In [1]:
import pandas as pd
from glob import glob
import itertools
from collections import Counter
import numpy as np
import numpy
import math
import matplotlib.pylab as plt
import os
import networkx as nx
from pypinyin import lazy_pinyin

In [37]:
# first put all the unique nodes together, second all pair combinations together,third pair combinations for every user
files=glob('./siteexample1/allusers/*')
for f in files:
    user=f.split('/')[-1].split('.')[0]
    userdata=pd.read_csv(f,names=['user_id','longtitude','latitude','app_name','app_type_name','start_time'])
    userdata[userdata['longtitude'].astype('str')!='nan']
    userdata=userdata[userdata['latitude'].astype('str')!='nan']
    userdata['location']=(userdata['longtitude'].astype('str')+','+userdata['latitude'].astype('str')).values
    userdata=userdata[userdata['app_name']!='其他']
    userdata=userdata[['user_id','location']]
    userdata=userdata.drop_duplicates()
    
    userdata.to_csv('./siteexample1/location/node.csv',header=False,mode='a+')
    
    app_list=userdata['location'].values
    pair=list(itertools.combinations(app_list,2))
    data=pd.DataFrame(pair)
    
    for i in range(len(data.values)):
        if data.values[i][0].split(',')[0]+data.values[0][0].split(',')[1]>data.values[i][1].split(',')[0]+data.values[0][0].split(',')[1]:
            data.values[i][0],data.values[i][1]=data.values[i][1],data.values[i][0]
    
    data.to_csv('./siteexample1/location/pair.csv',header=False,mode='a+')
    
    with open('./siteexample1/location/pair/' + str(user) +'.csv', 'a+') as f:
        data.to_csv(f,header=False)

In [39]:
# observed values
app_obs=pd.read_csv('./siteexample1/location/pair.csv',names=['node1','node2'],index_col=0)
app_obs['pair']=app_obs['node1'].astype('str')+' '+app_obs['node2'].astype('str')
counterdict=Counter(app_obs['pair'].values)
counterdict=pd.DataFrame(list(counterdict.items()),columns=['pair','app_obs'])
counterdict.to_csv('./siteexample1/location/observed_value.csv')    

In [40]:
# expected values
for i in range (10): # reconstruct random network for 10 times
    name='app_exp'+str(i)
    name=pd.read_csv('./siteexample1/location/node.csv',index_col=0,names=['user_id','app_name'])
    numpy.random.shuffle(name['app_name'].values) # shuffle the data
        
    users=name['user_id'].drop_duplicates()
    groups=name.groupby('user_id')
    
    path='./siteexample1/location/expected_value/'
    f=path+'app_exp'+str(i)+'.csv'

    for user in users:
        userdata=groups.get_group(user)
        userdata=userdata.drop_duplicates()

        app_list=userdata['app_name'].values
        pair=list(itertools.combinations(app_list,2))
        app_exp=pd.DataFrame(pair)
        app_exp.to_csv(f,header=0,mode='a+')

    app_exp=pd.read_csv(f,names=['node1','node2'])
    
    for i in range(len(app_exp.values)):
        if app_exp.values[i][0].split(',')[0]+app_exp.values[0][0].split(',')[1]>app_exp.values[i][1].split(',')[0]+app_exp.values[0][0].split(',')[1]:
            app_exp.values[i][0],app_exp.values[i][1]=app_exp.values[i][1],app_exp.values[i][0]
            
    app_exp['pair']=app_exp['node1'].astype('str')+' '+app_exp['node2'].astype('str')
    counterdict=Counter(app_exp['pair'].values)
    counterdict=pd.DataFrame(list(counterdict.items()),columns=['pair','app_obs'])
    counterdict.to_csv(f,header=0,mode='w')

In [41]:
# combine observed value and expected value
files=glob('./siteexample1/location/expected_value/*')

for f in files:
    name=f.split('/')[-1].split('.')[0]
    data=pd.read_csv(f,names=['pair','count'])
    diction=dict(data[['pair','count']].values)
    z_score=pd.read_csv('./siteexample1/location/observed_value.csv',index_col=0)
    z_score[name]=z_score['pair'].apply(lambda x: diction[x]
                                                    if x in diction
                                                    else numpy.nan)
    z_score.to_csv('./siteexample1/location/observed_value.csv')

/Users/xuhuimin/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [46]:
len(z_score[z_score['app_obs']!=1])

1330596

In [2]:
# z_score
z_score=pd.read_csv('./siteexample1/location/observed_value.csv',index_col=0)
z_score=z_score.fillna(0)
mean=z_score.iloc[:,[x for x in range(2,12)]].mean(axis=1)
std=z_score.iloc[:,[x for x in range(2,12)]].std(axis=1)
z_score['z_score']=(z_score['app_obs']-mean)/std
z_score.to_csv('./siteexample1/location/observed_value.csv')

/Users/xuhuimin/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
z_score=z_score[~z_score.isin([numpy.nan, numpy.inf, -numpy.inf]).any(1)]

In [50]:
z_score=pd.read_csv('./siteexample1/location/observed_value.csv',usecols=['pair','z_score'])
diction=dict(z_score.values)

files=glob('./siteexample1/location/pair/*')

for f in files:
    data=pd.read_csv(f,names=['node1','node2'])
    data['pair']=data['node1'].astype('str')+' '+data['node2'].astype('str')
    data['z_score']=data['pair'].apply(lambda x: diction[x]
                                  if x in diction
                                   else np.nan)
    data.to_csv(f)

In [51]:
# four categories group by z_score
files=glob('./siteexample1/location/pair/*')
lista=[]
listb=[]
listc=[]
for f in files:
    data=pd.read_csv(f)
    if len(data['z_score'])>0:
        user_id=f.split('/')[-1].split('.')[0]
        lista.append(user_id)
        a=numpy.array(data['z_score'].values)
        median=numpy.median(a)
        listb.append(median)
        pertentile_10=numpy.percentile(a,10)
        listc.append(pertentile_10)
df=pd.DataFrame([lista,listb,listc]).T
df.columns=['user_id','median','pertentile_10']
df.to_csv('./siteexample1/location/zscore_category.csv')

/Users/xuhuimin/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:4011: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)
/Users/xuhuimin/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:4269: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)
/Users/xuhuimin/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:4387: RuntimeWarning: invalid value encountered in multiply
  x2 = take(ap, indices_above, axis=axis) * weights_above
